# NN-Session-1 Solution: Binary Classification with Keras

This notebook provides complete solutions for the NN-session-1 exercises.
It demonstrates how to build a simple neural network for binary classification using Keras.

## 1. Setup and Library Imports

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn

# Machine learning tools
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# TensorFlow and Keras
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

## 2. Loading Preprocessed Sherlock Dataset

In [ ]:
# Load preprocessed features and labels
df2_features = pd.read_csv('sherlock/2apps_4f/sherlock_2apps_features.csv')
df2_labels = pd.read_csv('sherlock/2apps_4f/sherlock_2apps_labels.csv')

print(f"Features shape: {df2_features.shape}")
print(f"Labels shape: {df2_labels.shape}")
print(f"\nFeatures: {df2_features.columns.tolist()}")
print(f"\nFirst 5 rows of features:")
print(df2_features.head())
print(f"\nLabel distribution:")
print(df2_labels.value_counts())

In [ ]:
# SOLUTION: Split data into training and testing sets
train_F, test_F, train_L, test_L = train_test_split(df2_features, df2_labels, test_size=0.2, random_state=42)

print(f"Training set: {train_F.shape}")
print(f"Test set: {test_F.shape}")
print(f"\nTraining labels distribution:")
print(train_L.value_counts())
print(f"\nTest labels distribution:")
print(test_L.value_counts())

## 3. Building a One-Neuron Binary Classifier with Keras

In [ ]:
# SOLUTION: Define a function to create a one-neuron binary classifier
def NN_binary_clf(learning_rate=0.0003):
    """
    Create a one-neuron binary classifier using Keras.
    
    Parameters:
    -----------
    learning_rate : float
        Learning rate for the Adam optimizer
    
    Returns:
    --------
    model : Sequential
        Compiled Keras model
    """
    model = Sequential([
        Dense(1, activation='sigmoid', input_shape=(4,))
    ])
    
    # Create Adam optimizer with specified learning rate
    adam = Adam(learning_rate=learning_rate,
                beta_1=0.9, beta_2=0.999, amsgrad=False)
    
    # Compile the model
    model.compile(optimizer=adam,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

print("Function NN_binary_clf created successfully!")

In [ ]:
# SOLUTION: Create and display model architecture
model = NN_binary_clf(learning_rate=0.0003)

print("\nModel Architecture:")
print("="*60)
model.summary()
print("="*60)

## 4. Training the Model

In [ ]:
# SOLUTION: Train the model
print("Training the one-neuron model...")
print("="*60)

model_history = model.fit(train_F, train_L,
                          epochs=20, batch_size=32,
                          validation_data=(test_F, test_L),
                          verbose=1)

print("="*60)
print("Training completed!")

## 5. Analyzing Training History

In [ ]:
# SOLUTION: Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot loss
axes[0].plot(model_history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(model_history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Model Loss Over Epochs', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot accuracy
axes[1].plot(model_history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(model_history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Model Accuracy Over Epochs', fontsize=13, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTraining History Summary:")
print(f"Initial Training Loss: {model_history.history['loss'][0]:.4f}")
print(f"Final Training Loss: {model_history.history['loss'][-1]:.4f}")
print(f"Initial Validation Loss: {model_history.history['val_loss'][0]:.4f}")
print(f"Final Validation Loss: {model_history.history['val_loss'][-1]:.4f}")
print(f"\nInitial Training Accuracy: {model_history.history['accuracy'][0]:.4f}")
print(f"Final Training Accuracy: {model_history.history['accuracy'][-1]:.4f}")
print(f"Initial Validation Accuracy: {model_history.history['val_accuracy'][0]:.4f}")
print(f"Final Validation Accuracy: {model_history.history['val_accuracy'][-1]:.4f}")

## 6. Model Evaluation

In [ ]:
# SOLUTION: Evaluate Keras model
print("\n" + "="*60)
print("KERAS ONE-NEURON MODEL EVALUATION")
print("="*60)

test_loss, test_accuracy = model.evaluate(test_F, test_L, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Get predictions
test_pred_keras = (model.predict(test_F) > 0.5).astype(int).flatten()
print(f"\nConfusion Matrix:")
print(confusion_matrix(test_L, test_pred_keras))

## 7. Comparison with Traditional ML Models

In [ ]:
# SOLUTION: Helper function for model evaluation
def model_evaluate(model, test_F, test_L, model_name=""):
    """
    Evaluate a scikit-learn model.
    """
    test_L_pred = model.predict(test_F)
    acc = accuracy_score(test_L, test_L_pred)
    cm = confusion_matrix(test_L, test_L_pred)
    
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Confusion Matrix:")
    print(cm)
    
    return acc, cm

In [ ]:
# SOLUTION: Train Decision Tree model
print("\n" + "="*60)
print("DECISION TREE MODEL")
print("="*60)

model_dtc = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_split=8, random_state=42)
model_dtc.fit(train_F, train_L)

acc_dtc, cm_dtc = model_evaluate(model_dtc, test_F, test_L, "Decision Tree")

In [ ]:
# SOLUTION: Train Logistic Regression model
print("\n" + "="*60)
print("LOGISTIC REGRESSION MODEL")
print("="*60)

model_lr = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=42)
model_lr.fit(train_F, train_L)

acc_lr, cm_lr = model_evaluate(model_lr, test_F, test_L, "Logistic Regression")

## 8. Results Comparison

In [ ]:
# SOLUTION: Create comparison table
comparison_df = pd.DataFrame({
    'Model': ['One-Neuron NN (Keras)', 'Decision Tree', 'Logistic Regression'],
    'Test Accuracy': [test_accuracy, acc_dtc, acc_lr]
})

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(comparison_df.to_string(index=False))
print("="*60)

# Find best model
best_model_idx = comparison_df['Test Accuracy'].idxmax()
best_model = comparison_df.loc[best_model_idx, 'Model']
best_accuracy = comparison_df.loc[best_model_idx, 'Test Accuracy']

print(f"\nBest Model: {best_model}")
print(f"Best Accuracy: {best_accuracy:.4f}")

In [ ]:
# SOLUTION: Visualize model comparison
fig, ax = plt.subplots(figsize=(10, 6))

models = comparison_df['Model']
accuracies = comparison_df['Test Accuracy']
colors = ['steelblue', 'coral', 'lightgreen']

bars = ax.bar(models, accuracies, color=colors, edgecolor='black', linewidth=2, alpha=0.8)

ax.set_ylabel('Test Accuracy', fontsize=12)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_ylim([0, 1])
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bar, acc in zip(bars, accuracies):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.4f}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.xticks(rotation=15, ha='right')
plt.tight_layout()
plt.show()

## 9. Key Findings and Discussion

### Observations:

1. **Model Performance**: The one-neuron Keras model achieves comparable accuracy to traditional ML models.

2. **Training Dynamics**: 
   - The Keras model shows convergence over epochs
   - Validation accuracy stabilizes after several epochs
   - No significant overfitting observed

3. **Comparison with Traditional ML**:
   - Decision Tree and Logistic Regression provide baseline performance
   - One-neuron NN is essentially equivalent to logistic regression
   - All models achieve similar accuracy on this dataset

### Why Validation Metrics Matter:

- **Training metrics** show how well the model fits the training data
- **Validation metrics** show how well the model generalizes to unseen data
- Validation metrics are more important for assessing true model performance
- Large gap between training and validation metrics indicates overfitting

### Ways to Improve the One-Neuron Model:

1. **Add Hidden Layers**: Increase model capacity with hidden layers
2. **Adjust Learning Rate**: Fine-tune the learning rate for better convergence
3. **Increase Epochs**: Train for more epochs to improve convergence
4. **Batch Size Tuning**: Experiment with different batch sizes
5. **Regularization**: Add L1/L2 regularization to prevent overfitting
6. **Feature Engineering**: Create more discriminative features

### Cybersecurity Application:

This binary classification approach can be used for:
- **Malware Detection**: Classify applications as benign or malicious
- **Anomaly Detection**: Identify unusual application behavior
- **Real-time Monitoring**: Classify running applications on mobile devices
- **Threat Intelligence**: Build profiles of known malicious applications